In [9]:
#Importando librerías necesarias 

import psycopg2
from psycopg2.extras import execute_values
import requests
import pandas as pd
from datetime import datetime
import datetime as dt

In [10]:
#Estableciendo la conexión con la base de datos 

with open("C:/Users/Carolina/Desktop/password.txt",'r') as file:
    pwd= file.read()
host='data-engineer-cluster.cyhh5bfevlmn.us-east-1.redshift.amazonaws.com'
db='data-engineer-database'
user='carolina_cargnelutti_coderhouse'

try: 
    conn = psycopg2.connect(
        host=host,
        database=db,
        password=pwd,
        port=5439,
        user=user      
  )
    cursor = conn.cursor()
    print('Se realizó la conexión correctamente ')

except Exception as e:
    print('No fue posible realizar la conexión')

Se realizó la conexión correctamente 


In [11]:
#Creando la tabla en la base de datos

query="CREATE TABLE IF NOT EXISTS carolina_cargnelutti_coderhouse.weather(id INT primary key not null, location VARCHAR NOT NULL,region VARCHAR,temperature INTEGER,condition VARCHAR,wind INT,wind_direction VARCHAR,humidity INT,feels_like INTEGER ,visibility INTEGER,updated DATETIME, date_transfered DATETIME)"
cursor.execute(query)
conn.commit()

In [12]:
#Comprobando que la tabla esta vacía. En el caso de tener registros, estos se eliminan:

cursor.execute('SELECT * FROM carolina_cargnelutti_coderhouse.weather')
weather_table=cursor.fetchall()
if len(weather_table)>0:
    cursor.execute('TRUNCATE TABLE carolina_cargnelutti_coderhouse.weather')
    print('Se eliminaron correctamente todos los registros de la tabla')
else:
    print('La tabla no tiene registros')

La tabla no tiene registros


In [13]:
#Extrayendo datos de la API

counter=1

def request(localidad):
    url='http://api.weatherapi.com/v1/forecast.json?key=9bc72106866d42c28da193618240701 &q='+str(localidad)+'&days=1&aqi=yes&alerts=no'
    response=requests.get(url).json()
    return response

def dateFormat(date):
   date_format = '%Y-%m-%d %H:%M' 
   update_date = datetime.strptime(date, date_format)
   return update_date

date_transfered=dt.datetime.today()

data={'id':[],'location':[],'region':[],'temperature':[],'condition':[], 'wind':[], 'wind_direction':[], 'humidity':[],
     'feels_like': [], 'visibility':[],'updated':[], 'date_transfered': []}

locations= ['Buenos Aires','Catamarca','Córdoba','Cataratas del Iguazú','Comodoro Rivadavia','Concordia','El Calafate','Ezeiza','Formosa','Jujuy','Mar del Plata','Mendoza','Neuquen','Paraná','Posadas','Puerto Madryn',
    'Reconquista','Salta','San Carlos de Bariloche', 'Rio Gallegos', 'Termas de Rio Hondo', 'Ushuaia']

for location in locations:
    response=request(location)
    location= response['location']['name']
    region= response['location']['region']
    temperature =int(response['current']['temp_c'])
    feels_like= int(response['current']['feelslike_c'])
    condition = response['current']['condition']['text']
    wind = int(response['current']['wind_kph'])
    wind_direction=response['current']['wind_dir']
    humidity=int(response['current']['humidity'])
    visibility=int(response['current']['vis_km'])
    updated=dateFormat(response['current']['last_updated'])

    data['id'].append(counter)
    data['location'].append(location)
    data['region'].append(region)
    data['temperature'].append(temperature)
    data['condition'].append(condition)
    data['wind'].append(wind)
    data['wind_direction'].append(wind_direction)
    data['humidity'].append(humidity)
    data['feels_like'].append(feels_like)
    data['visibility'].append(visibility)
    data['updated'].append(updated)
    data['date_transfered'].append(date_transfered)
    
    counter = counter + 1
df=pd.DataFrame(data)

In [14]:
#Limpieza del dataframe:Eliminando datos nulos y duplicados:
df.dropna()
df.drop_duplicates(subset='location')

df.head(5)

,id,location,region,temperature,condition,wind,wind_direction,humidity,feels_like,visibility,updated,date_transfered
0,1,Buenos Aires,Distrito Federal,16,Clear,4,N,94,16,10,2024-04-20 04:15:00,2024-04-20 09:31:07.722081
1,2,Catamarca,Catamarca,13,Clear,3,NNW,66,13,10,2024-04-20 04:15:00,2024-04-20 09:31:07.722081
2,3,Córdoba,Cordoba,12,Clear,6,NNW,88,12,10,2024-04-20 04:30:00,2024-04-20 09:31:07.722081
3,4,Cataratas Del Iguazu,Misiones,16,Clear,11,E,82,16,10,2024-04-20 04:15:00,2024-04-20 09:31:07.722081
4,5,Comodoro Rivadavia,Distrito Federal,10,Clear,22,W,50,6,10,2024-04-20 04:15:00,2024-04-20 09:31:07.722081


In [15]:
#Actualizando la fecha de inserción de los datos e insertándolos en la base de datos:

df.date_transfered=dt.datetime.today()
execute_values(cursor,
'''
INSERT INTO carolina_cargnelutti_coderhouse.weather (id,location,region,temperature,condition,wind,wind_direction,humidity,feels_like,visibility,updated, date_transfered)
VALUES %s
''',[tuple(x) for x in df.values],page_size=len(df))
conn.commit()

In [16]:
cursor.close()
conn.close()